In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Setup plotting
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
# Set Matplotlib defaults
plt.rc('figure', autolayout=True)
plt.rc('axes', labelweight='bold', labelsize='large',
       titleweight='bold', titlesize=18, titlepad=10)
plt.rc('animation', html='html5')

/kaggle/input/sri-lanka-weather-dataset/SriLanka_Weather_Dataset.csv


/tmp/ipykernel_33/1096943364.py:18: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-whitegrid')


In [3]:
#Extracting the medical insurance data
weather = pd.read_csv('../input/sri-lanka-weather-dataset/SriLanka_Weather_Dataset.csv')
weather.head(5)
cities = pd.DataFrame({'Cities' : weather.city.unique()})
cities

,Cities
0,Colombo
1,Mount Lavinia
2,Kesbewa
3,Moratuwa
4,Maharagama
5,Ratnapura
6,Kandy
7,Negombo
8,Sri Jayewardenepura Kotte
9,Kalmunai


# Visualizing Data for Kandy

In [4]:
cities = weather.city.unique()
#collecting data for kandy
weather_sl_kandy = weather[weather['city'] == 'Kandy']
weather_sl_kandy.head()

,time,weathercode,temperature_2m_max,temperature_2m_min,temperature_2m_mean,apparent_temperature_max,apparent_temperature_min,apparent_temperature_mean,sunrise,sunset,...,precipitation_hours,windspeed_10m_max,windgusts_10m_max,winddirection_10m_dominant,et0_fao_evapotranspiration,latitude,longitude,elevation,country,city
29496,2010-01-01,2,25.3,18.7,22.0,27.2,20.3,23.8,2010-01-01T00:49,2010-01-01T12:32,...,0.0,15.3,38.9,57,3.43,7.300003,80.600006,510.0,Sri Lanka,Kandy
29497,2010-01-02,2,25.6,19.4,22.2,28.4,21.6,24.8,2010-01-02T00:50,2010-01-02T12:32,...,0.0,13.3,33.1,46,2.95,7.300003,80.600006,510.0,Sri Lanka,Kandy
29498,2010-01-03,51,25.0,19.5,22.1,28.2,22.1,25.0,2010-01-03T00:50,2010-01-03T12:33,...,1.0,10.7,28.1,40,2.84,7.300003,80.600006,510.0,Sri Lanka,Kandy
29499,2010-01-04,51,25.3,19.0,21.9,27.7,21.3,24.3,2010-01-04T00:50,2010-01-04T12:33,...,2.0,10.4,29.2,23,3.16,7.300003,80.600006,510.0,Sri Lanka,Kandy
29500,2010-01-05,1,25.0,16.3,20.8,28.0,17.5,22.6,2010-01-05T00:51,2010-01-05T12:34,...,0.0,9.7,27.0,28,4.18,7.300003,80.600006,510.0,Sri Lanka,Kandy


In [8]:
#changing date format
def dateTime(row):
    new_time = pd.to_datetime(row.time)
    row['time'] = new_time
    return row

weatherDate = weather_sl_kandy.apply(dateTime, axis='columns')
#dropping unwanted, redundant columns
weather_kandy = weatherDate.drop(columns = ['country', 'temperature_2m_mean', 'sunrise', 'sunset','apparent_temperature_mean','snowfall_sum', 'longitude', 'latitude', 'elevation'])
weather_kandy['year'] = weather_kandy['time'].dt.year
weather_kandy['month'] = weather_kandy['time'].dt.month
weather_kandy['day'] = weather_kandy['time'].dt.day
weather_kandy = weather_kandy.drop(columns = ['time'])
weather_kandy.reset_index(inplace=True, drop=True)
col = weather_kandy.pop('day')
weather_kandy.insert(0, col.name, col)
col2 = weather_kandy.pop('month')
weather_kandy.insert(0, col2.name, col2)
col3 = weather_kandy.pop('year')
weather_kandy.insert(0, col3.name, col3)
weather_kandy

,year,month,day,weathercode,temperature_2m_max,temperature_2m_min,apparent_temperature_max,apparent_temperature_min,shortwave_radiation_sum,precipitation_sum,rain_sum,precipitation_hours,windspeed_10m_max,windgusts_10m_max,winddirection_10m_dominant,et0_fao_evapotranspiration,city
0,2010,1,1,2,25.3,18.7,27.2,20.3,18.09,0.0,0.0,0.0,15.3,38.9,57,3.43,Kandy
1,2010,1,2,2,25.6,19.4,28.4,21.6,15.47,0.0,0.0,0.0,13.3,33.1,46,2.95,Kandy
2,2010,1,3,51,25.0,19.5,28.2,22.1,14.99,0.1,0.1,1.0,10.7,28.1,40,2.84,Kandy
3,2010,1,4,51,25.3,19.0,27.7,21.3,16.55,0.6,0.6,2.0,10.4,29.2,23,3.16,Kandy
4,2010,1,5,1,25.0,16.3,28.0,17.5,22.68,0.0,0.0,0.0,9.7,27.0,28,4.18,Kandy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4911,2023,6,13,53,27.2,22.2,30.9,25.2,18.17,3.1,3.1,9.0,16.6,45.7,221,3.51,Kandy
4912,2023,6,14,53,26.9,22.5,30.3,25.3,18.26,4.0,4.0,12.0,16.8,42.5,218,3.56,Kandy
4913,2023,6,15,51,28.0,22.5,31.6,25.3,17.12,0.6,0.6,4.0,14.2,37.1,217,3.53,Kandy
4914,2023,6,16,53,27.7,22.2,31.6,25.1,21.14,1.9,1.9,9.0,17.9,45.4,224,4.09,Kandy
